In [11]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [12]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report


In [13]:
train_df = pd.read_csv('/content/twitter_training.csv', encoding='ISO-8859-1')
valid_df = pd.read_csv('/content/twitter_validation.csv', encoding='ISO-8859-1')

train_df.columns = ['ID', 'Entity', 'Sentiment', 'Text']
valid_df.columns = ['ID', 'Entity', 'Sentiment', 'Text']

df = pd.concat([train_df, valid_df], ignore_index=True)
df.head()


,ID,Entity,Sentiment,Text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [15]:
df.dropna(inplace=True)

def preprocess_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    text = re.sub(r'#[A-Za-z0-9_]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()

    tokens = nltk.word_tokenize(text)
    stopwords_list = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stopwords_list]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return ' '.join(tokens)

df['clean_text'] = df['Text'].apply(preprocess_text)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_text'], df['Sentiment'], test_size=0.2, random_state=42
)


In [17]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [18]:
svm_classifier = LinearSVC()
svm_classifier.fit(X_train_tfidf, y_train)


LinearSVC()

In [19]:
y_pred = svm_classifier.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.6955797053136876
              precision    recall  f1-score   support

  Irrelevant       0.68      0.55      0.61      2636
    Negative       0.69      0.80      0.74      4439
     Neutral       0.70      0.64      0.67      3681
    Positive       0.70      0.73      0.72      4243

    accuracy                           0.70     14999
   macro avg       0.69      0.68      0.68     14999
weighted avg       0.70      0.70      0.69     14999



In [20]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00


In [26]:
def predict_sentiment(tweet):
    processed_tweet = preprocess_text(tweet)
    tweet_tfidf = tfidf_vectorizer.transform([processed_tweet])

    prediction = svm_classifier.predict(tweet_tfidf)[0]
    return f"{prediction}"


In [27]:
import gradio as gr

interface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(label="Enter a Tweet"),
    outputs=gr.Textbox(label="Sentiment Prediction"),
    title="Twitter Sentiment Analysis",
    description="Enter a tweet and get its sentiment: Positive, Neutral, Negative or Irrelevant."
)

interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ad481817a00423d58b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
